In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./
%config Completer.use_jedi = False
%load_ext Cython
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [2]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:746:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization_Cyth

In [3]:
URM_all_dataframe = pd.read_csv('/kaggle/input/urm-true-binary/URM_True_Binary.csv')
URM_all_dataframe
URM_all = sps.coo_matrix((URM_all_dataframe["Data"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)
URM_all

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [4]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# split data into train and validation data 80/20
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [5]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 296 ( 0.7%) Users that have less than 1 test interactions


In [6]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [7]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [8]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(epochs = 500, l1_ratio = 0.001, alpha = 0.01, 
               positive_only = True, topK = 384)

SLIMElasticNetRecommender: Processed 3087 (12.6%) in 5.00 min. Items per second: 10.29
SLIMElasticNetRecommender: Processed 6619 (27.0%) in 10.00 min. Items per second: 11.03
SLIMElasticNetRecommender: Processed 10073 (41.1%) in 15.00 min. Items per second: 11.19
SLIMElasticNetRecommender: Processed 13582 (55.4%) in 20.00 min. Items per second: 11.32
SLIMElasticNetRecommender: Processed 17081 (69.7%) in 25.00 min. Items per second: 11.38
SLIMElasticNetRecommender: Processed 20616 (84.1%) in 30.00 min. Items per second: 11.45
SLIMElasticNetRecommender: Processed 24094 (98.3%) in 35.00 min. Items per second: 11.47
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 35.57 min. Items per second: 11.48


In [9]:
RP3beta_all = RP3betaRecommender(URM_train)
RP3beta_all.fit(alpha= 1.0, beta= 0.28666076265452467, topK= 57, implicit= True)

RP3betaRecommender: Similarity column 24507 (100.0%), 2256.46 column/sec. Elapsed time 10.86 sec


In [10]:
EASE_R = EASE_R_Recommender(URM_all)
#%%
EASE_R.fit(topK = 416, l2_norm = 115.67139771839786, normalize_matrix = False)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 10.58 min


In [11]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
recommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_all)
recommender_SLIM_BPR_Cython.fit(epochs= 615, sgd_mode= 'sgd', topK= 49, lambda_i= 0.0001, lambda_j= 0.0017579136035800475, learning_rate= 0.032671047315169746)

SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 28458.00 MB (88.63%) of 32110.00 MB, required is 2402.37 MB. Using dense matrix.
Processed 41629 (100.0%) in 1.01 sec. BPR loss is 1.67E-01. Sample per second: 41194
SLIM_BPR_Recommender: Epoch 1 of 615. Elapsed time 0.29 sec
Processed 41629 (100.0%) in 0.29 sec. BPR loss is 6.09E-01. Sample per second: 145202
SLIM_BPR_Recommender: Epoch 2 of 615. Elapsed time 0.56 sec
Processed 41629 (100.0%) in 0.57 sec. BPR loss is 1.03E+00. Sample per second: 72429
SLIM_BPR_Recommender: Epoch 3 of 615. Elapsed time 0.85 sec
Processed 41629 (100.0%) in 0.87 sec. BPR loss is 1.49E+00. Sample per second: 48045
SLIM_BPR_Recommender: Epoch 4 of 615. Elapsed time 1.14 sec
Processed 41629 (100.0%) in 1.15 sec. BPR loss is 1.84E+00. Sample per second: 36110
SLIM_BPR_Recommender: Epoch 5 of 615. Elapsed time 1.43 sec
Processed 41629 (100.0%) in 0.44 sec. BPR loss is 2.23E+00. Sample per second: 95618
SLIM_BPR_Recommender: Epoc

In [12]:
print("MAP of the starting models")

result_df, _ = evaluator_valid.evaluateRecommender(recommender_SLIMElasticNet)
print("SLIM - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(RP3beta_all)
print("RP3beta - MAP: {}".format(result_df.loc[10]["MAP"]))

MAP of the starting models
EvaluatorHoldout: Processed 41333 (100.0%) in 40.24 sec. Users per second: 1027
SLIM - MAP: 0.03458570998460277
EvaluatorHoldout: Processed 41333 (100.0%) in 25.03 sec. Users per second: 1652
RP3beta - MAP: 0.03392776209024571


In [13]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of four predictions scores
    R = R1*alpha + R2*beta + R3*theta + R3*(1-alpha-beta-theta)
    
    Class from Dacrema exercise modified by Antonio Ercolani
    The original took as input 2 recommender

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3, recommender_4):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        self.recommender_4 = recommender_4
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5, theta = 0):

        self.alpha = alpha
        self.beta = beta
        self.theta = theta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)
        item_weights_4 = self.recommender_4._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        norm_item_weights_4 = LA.norm(item_weights_4, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_4 == 0:
            raise ValueError("Norm {} of item weights for recommender 4 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * self.theta + item_weights_4 / norm_item_weights_4 * (1-self.alpha-self.beta-self.theta)

        return item_weights

In [14]:
import random

recommender_object = DifferentLossScoresHybridRecommender(URM_train, recommender_SLIMElasticNet, RP3beta_all, EASE_R, recommender_SLIM_BPR_Cython)

best_model = {
    "MAP" : 0,
    "alpha" : 0,
    "beta": 0,
    "theta": 0,
    "norm" : 0
}

n_searches = 0

for norm in [1,2]:
    while n_searches<20:
        alpha = random.uniform(0, 1)
        beta = random.uniform(0, 1-alpha)
        theta = random.uniform(0, 1- (alpha+beta))
        print("----")
        recommender_object.fit(norm, alpha, beta, theta)

        result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
        print("Norm: {}, Alpha: {}, Beta: {}, Theta: {}, Gamma: {}, Result: {}".format(norm, alpha, beta, theta, 1-(alpha+beta+theta), result_df.loc[10]["MAP"]))

        if result_df.loc[10]["MAP"] > best_model["MAP"]:
            best_model["MAP"] = result_df.loc[10]["MAP"]
            best_model["alpha"] = alpha
            best_model["beta"] = beta
            best_model["theta"] = theta
            best_model["norm"] = norm
                
        n_searches += 1
print("----")
print("Best model has MAP: {} with alpha: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["norm"]))


----
EvaluatorHoldout: Processed 41333 (100.0%) in 1.23 min. Users per second: 560
Norm: 1, Alpha: 0.4481676635396029, Beta: 0.34490840535423567, Theta: 0.08754136378292977, Gamma: 0.11938256732323171, Result: 0.03813008764663454
----
EvaluatorHoldout: Processed 41333 (100.0%) in 1.23 min. Users per second: 558
Norm: 1, Alpha: 0.2064685472454021, Beta: 0.7928179637119308, Theta: 0.0002642262777013883, Gamma: 0.0004492627649657077, Result: 0.034650219913446266
----
EvaluatorHoldout: Processed 41333 (100.0%) in 1.24 min. Users per second: 556
Norm: 1, Alpha: 0.3975510044964625, Beta: 0.2846439048040098, Theta: 0.08862794490606411, Gamma: 0.2291771457934636, Result: 0.04223097997333898
----
EvaluatorHoldout: Processed 41333 (100.0%) in 1.26 min. Users per second: 549
Norm: 1, Alpha: 0.9468944546049125, Beta: 0.04751570022842514, Theta: 0.004504226926927876, Gamma: 0.0010856182397345204, Result: 0.034908221226052985
----
EvaluatorHoldout: Processed 41333 (100.0%) in 1.26 min. Users per sec

In [15]:
recommender = DifferentLossScoresHybridRecommender(URM_all, recommender_SLIMElasticNet, RP3beta_all, EASE_R, recommender_SLIM_BPR_Cython)
recommender.fit(1, 0.07510083706775092, 0.034152002358325294, 0.6216734083665473)

In [16]:
test_users = pd.read_csv('/kaggle/input/competition-data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
41111,41624
41112,41625
41113,41626
41114,41627


In [17]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10, remove_seen_flag = True))
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission4.csv', index=False)